# Exame CSC-35

### Grupo

- Alexandre Bellargus
- Pedro Igor
- Stelios

## Definições

In [ ]:
import random
import pygame
from math import cos, sin, exp, atan2, pi

WIDTH = 1800
HEIGHT = 1000
WIFI_RANGE = 300

IN_ATTACK = False

## Nó de Rede

In [ ]:
# Imports
import aodv

# Class Definition
class node():

    # Constructor
    def __init__(self):
        self.node_id = ""
        self.aodv = None

    # Main routine
    def main(self, n, node_id):
        
        # Store the Node ID
        self.node_id = node_id
        
        # Initialize and start the protocol handler thread
        self.aodv = aodv.aodv()
        self.aodv.set_node_id(node_id)
        self.aodv.set_node_count(n)
        self.aodv.start()

## Definição dos Drones

In [ ]:
class Drone:
    def __init__(self, x, y, i, numDrones):
        self.x = random.randint(0, x)
        self.y = random.randint(0, y)
        self.angle = random.randint(0, 360)
        self.scalar = 4
        self.velocity = (self.scalar * cos(self.angle), self.scalar * sin(self.angle))
        self.node_id = "n" + str(i + 1)
        self.droneNode = node()
        self.droneNode.main(numDrones, self.node_id)
        self.dronesList = ['n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14', 'n15', 'n16', 'n17', 'n18', 'n19', 'n20']
        self.dronesList = self.dronesList[:numDrones]
        self.dronesList.remove(self.node_id)
        self.attacker = False
        # self.linkAllAodv()

    def update(self):
        self.updatePosition()

    def linkAllAodv(self):
        self.droneNode.aodv.aodv_add_neighbors(self.dronesList)

    def updatePosition(self):
        self.x += self.velocity[0]
        self.y += self.velocity[1]

        if self.x < 0:
            self.velocity = (abs(self.velocity[0]), self.velocity[1])
        elif self.x > WIDTH:
            self.velocity = (-abs(self.velocity[0]), self.velocity[1])

        if self.y < 0:
            self.velocity = (self.velocity[0], abs(self.velocity[1]))
        elif self.y > HEIGHT:
            self.velocity = (self.velocity[0], -abs(self.velocity[1]))

    def checkAllClose(self, droneList, distance):
        self.checkConection(droneList, distance)

        minDistance = 100000
        saveX = 0
        saveY = 0
        postitions = self.droneNode.aodv.aodv_get_neighbors_positions().copy()

        if self.attacker:
            saveX = 500
            saveY = 300
            minDistance = ((self.x - saveX)**2 + (self.y - saveY)**2)**0.5
            if minDistance < distance:
                self.angle = self.getAngle(saveX, saveY)
                self.velocity = (-self.scalar * cos(self.angle), -self.scalar * sin(self.angle))

            if minDistance > distance:
                self.angle = self.getAngle(saveX, saveY)
                self.velocity = (self.scalar * cos(self.angle), self.scalar * sin(self.angle))

            return

        if IN_ATTACK:
            for drone in droneList:
                if self.node_id != drone.node_id and drone.node_id in postitions.keys():
                    minDrone = postitions[drone.node_id] 
                    distanceToDrone = self.getDistanceXY(int(minDrone[0]), int(minDrone[1]))
                    if distanceToDrone < minDistance:
                        minDistance = distanceToDrone
                        saveX = int(minDrone[0])
                        saveY = int(minDrone[1])

        else:
            for drone in droneList:
                if self.node_id != drone.node_id:
                    distanceToDrone = self.getDistanceXY(drone.x, drone.y)
                    if distanceToDrone < minDistance:
                        minDistance = distanceToDrone
                        saveX = drone.x
                        saveY = drone.y

        if self.getMinDistFromWalls() < distance:
            self.angle = self.getAngleFromWalls()
            self.velocity = (self.scalar * cos(self.angle), self.scalar * sin(self.angle))
            return

        if minDistance < distance:
            self.angle = self.getAngle(saveX, saveY)
            self.velocity = (-self.scalar * cos(self.angle), -self.scalar * sin(self.angle))

        if minDistance > distance:
            self.angle = self.getAngle(saveX, saveY)
            self.velocity = (self.scalar * cos(self.angle), self.scalar * sin(self.angle))

        

    def checkConection(self, droneList, distance):
        for drone in droneList:
            distanceToDrone = self.getDistance(drone)
            prob = exp(-distanceToDrone / (distance//5))
            if drone.node_id not in self.droneNode.aodv.aodv_get_active_routes() and prob > random.random():
                self.droneNode.aodv.aodv_add_neighbors([drone.node_id])
            self.droneNode.aodv.probabilities_table[drone.node_id] = exp(-self.getDistance(drone) / (distance))
            

    def getDistance(self, drone):
        return ((self.x - drone.x)**2 + (self.y - drone.y)**2)**0.5
    
    def getDistanceXY(self, x, y):
        return ((self.x - x)**2 + (self.y - y)**2)**0.5
    
    def getVector(self, x, y):
        return (x - self.x, y - self.y)
    
    def getAngle(self, x, y):
        vector = self.getVector(x, y)
        return atan2(vector[1], vector[0])
    
    def getMinDistFromWalls(self):
        return min(self.x, self.y, WIDTH - self.x, HEIGHT - self.y)
    
    def getAngleFromWalls(self):
        #check the closest wall
        dist = self.getMinDistFromWalls()
        if dist == self.x:
            return 0
        elif dist == self.y:
            return pi/2
        elif dist == WIDTH - self.x:
            return pi
        elif dist == HEIGHT - self.y:
            return 3*pi/2

## Criação dos Drones

In [ ]:
from getpass import getpass

command = "echo 'test'"
!echo {getpass()} | sudo -S {command}

numDrones = 20

firstDrone = None

drones = []
for i in range(numDrones):
    newDrone = Drone(WIDTH, HEIGHT, i, numDrones)
    drones.append(newDrone)
    if firstDrone == None:
        firstDrone = newDrone

## Funções Auxiliares

In [ ]:
def get_drone_by_node_id(node_id):
    for drone in drones:
        if drone.node_id == node_id:
            return drone

def print_drone_routing_table(drone, screen, font, name = "n1"):
    table = drone.droneNode.aodv.aodv_get_routing()
    text = "Tabela de Rotas de " + name
    topTextSurface = font.render(text, True, (0, 0, 0))
    topTextRect = topTextSurface.get_rect()
    topTextRect.topleft = (0, 0)
    screen.blit(topTextSurface, topTextRect)
    text = "{:15} {:15} {:10} {:10} {:10}".format("Destination", "Next-Hop", "Seq-No", "Hop-Count", "Status")
    topTextSurface = font.render(text, True, (0, 0, 0))
    topTextRect = topTextSurface.get_rect()
    topTextRect.topleft = (0, 20)
    screen.blit(topTextSurface, topTextRect)

    line = 1    
    for r in table.values():
        line += 1
        text = f"{r['Destination']}               {r['Next-Hop']}           {r['Seq-No']}                {r['Hop-Count']}            {r['Status']}"
        textSurface = font.render(text, True, (0, 0, 0))
        textRect = textSurface.get_rect()
        textRect.topleft = (0, line * 20)
        screen.blit(textSurface, textRect)

def attack_drone(drone, time, x, y):
    drone.attacker = True
    for dest in drone.dronesList:
        for dron in drone.dronesList:
            message = dron + "," + str(int(x)) + "," + str(int(y)) + ",100000"
            drone.droneNode.aodv.aodv_send_message_custom(drone.node_id, dest, message)

def send_neighbors_postitions(drone, time):
    if drone.attacker:
        attack_drone(drone, time, 500, 300)
        for name in drone.dronesList:
            message = drone.node_id + "," + str(500) + "," + str(300) + ",100000"
            drone.droneNode.aodv.aodv_send_message_custom(drone.node_id, name, message)
        return
    for name in drone.dronesList:
        message = drone.node_id + "," + str(int(drone.x)) + "," + str(int(drone.y)) + "," + str(time)
        drone.droneNode.aodv.aodv_send_message_custom(drone.node_id, name, message)

def print_drone_neighbors_positions(drone, screen, font):
    postitions = drone.droneNode.aodv.aodv_get_neighbors_positions().copy()
    text = "Posições Conhecidas dos Drones"
    topTextSurface = font.render(text, True, (0, 0, 0))
    # draw in the left upper corner
    topTextRect = topTextSurface.get_rect()
    topTextRect.topright = (WIDTH, 0)
    screen.blit(topTextSurface, topTextRect)
    text = "{:15} {:10} {:10}".format("ID", "X", "Y")
    topTextSurface = font.render(text, True, (0, 0, 0))
    # draw in the left upper corner
    topTextRect = topTextSurface.get_rect()
    topTextRect.topright = (WIDTH, 20)
    screen.blit(topTextSurface, topTextRect)
    line = 1
    for d in postitions.keys():
        line += 1
        dron = postitions[d]
        text = "{:15} {:10} {:10}".format(d, str(dron[0]), str(dron[1]))
        textSurface = font.render(text, True, (0, 0, 0))
        textRect = textSurface.get_rect()
        textRect.topright = (WIDTH, line * 20)
        screen.blit(textSurface, textRect)


## Criação da Interface Gráfica

In [ ]:
# %%capture
import warnings
warnings.filterwarnings('ignore')
pygame.init()

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Drones")

clock = pygame.time.Clock()

FPS = 6

drones1 = drones[0:numDrones//3]
drones2 = drones[numDrones//3:2*numDrones//3]
drones3 = drones[2*numDrones//3:3*numDrones//3]

droneImage = pygame.image.load("drone.png")
droneImage = pygame.transform.scale(droneImage, (50, 50))

plataformaImage = pygame.image.load("plataforma.png")
plataformaImage = pygame.transform.scale(plataformaImage, (500, 500))

for drone in drones:
    drone.update()
    drone.checkAllClose(drones, WIFI_RANGE)

frame = 1
droneAtual = 0

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE):
            # firstDrone.droneNode.aodv.aodv_show_routing_table(False)
            pygame.quit()
            raise SystemExit
    
    if event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
        drones[1].attacker = True
        IN_ATTACK = True

    # Quando clica com a seta esquerda
    if event.type == pygame.KEYDOWN and event.key == pygame.K_LEFT:
        droneAtual -= 1
        droneAtual = droneAtual % len(drones)
    
    # Quando clica com a seta direita
    if event.type == pygame.KEYDOWN and event.key == pygame.K_RIGHT:
        droneAtual += 1
        droneAtual = droneAtual % len(drones)


    screen.fill((167, 199, 231))

    screen.blit(plataformaImage, (WIDTH/2 - 250, HEIGHT/2 - 250))

    if frame % (3*FPS) == 0:
        for drone in drones1:
            send_neighbors_postitions(drone, frame//FPS)
    
    if (frame + FPS) % (3*FPS) == 0:
        for drone in drones2:
            send_neighbors_postitions(drone, frame//FPS)
    
    if (frame + 2*FPS) % (3*FPS) == 0:
        for drone in drones3:
            send_neighbors_postitions(drone, frame//FPS)

    if IN_ATTACK:
        surface1 = pygame.Surface((WIFI_RANGE,WIFI_RANGE))
        surface1.set_colorkey((0,0,0))
        surface1.set_alpha(100)
        pygame.draw.circle(surface1, (50,50,50), (WIFI_RANGE/2, WIFI_RANGE/2), WIFI_RANGE/2)
        screen.blit(surface1, (500 - WIFI_RANGE/2, 300 - WIFI_RANGE/2))

    for drone in drones:
        drone.update()
        drone.checkAllClose(drones, WIFI_RANGE)
        surface1 = pygame.Surface((WIFI_RANGE,WIFI_RANGE))
        surface1.set_colorkey((0,0,0))
        surface1.set_alpha(100)
        if drone.attacker:
            pygame.draw.circle(surface1, (255,0,0), (WIFI_RANGE/2, WIFI_RANGE/2), WIFI_RANGE/2)
        else:
            pygame.draw.circle(surface1, (0,0,255), (WIFI_RANGE/2, WIFI_RANGE/2), WIFI_RANGE/2)
        screen.blit(surface1, (drone.x - WIFI_RANGE/2, drone.y - WIFI_RANGE/2))

        active_routes = drone.droneNode.aodv.aodv_get_active_routes()
        for key in active_routes:
            # desenhar uma linha verde ligando os dois drones
            other_drone = get_drone_by_node_id(key)
            pygame.draw.line(screen, (0, 255, 0), (drone.x, drone.y), (other_drone.x, other_drone.y), 3)

        for key in drone.droneNode.aodv.aodv_get_inactive_routes():
            if key in active_routes:
                continue
            # desenhar uma linha verde ligando os dois drones
            other_drone = get_drone_by_node_id(key)
            pygame.draw.line(screen, (70, 30, 30), (drone.x, drone.y), (other_drone.x, other_drone.y), 1)

    for drone in drones:
        font = pygame.font.Font(None, 20)
        text = font.render(str(drone.node_id) + " " + str(int(drone.x)) + " " + str(int(drone.y)), True, (255, 255, 255))
        screen.blit(text, (drone.x + 10, drone.y))

        screen.blit(droneImage, (drone.x - 25, drone.y - 25))

        # pygame.draw.circle(screen, (255, 255, 255), (drone.x, drone.y), 7)
        if drone.attacker:
            pygame.draw.circle(screen, (255, 0, 0), (drone.x, drone.y), 7)

    print_drone_routing_table(drones[droneAtual], screen, font, f"n{droneAtual+1}")
    print_drone_neighbors_positions(drones[droneAtual], screen, font)

    pygame.display.flip()
    clock.tick(6)

    frame += 1